In [1]:
pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/76/e8/260752c1bc8dc856b27cfefaa3fca29ba02a15f649bbf6e9c15ed54e5480/xgboost-2.0.2-py3-none-macosx_12_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import sklearn
import xgboost

In [5]:
df = pd.read_csv("../clean_data/final_df.csv", index_col=0)

In [6]:
df.head()

,Company,Device,Market,SubMarket,Year,Value,Region,Spendings,DevSpends,MarketSpends,SubMarketSpends,Cogs,Cogs_1,Cogs_2,Revenue,Revenue_1,Revenue_2,Ebitda,Ebitda_1,Ebitda_2
0,ABB,Amplifier/Comparator,Industrial,Power & Energy,2018,7.466000,AmericasAsia & Oceania (exc. Japan)EMEAJapan,200.530,172603.835,744408.882,59122.300,19059.0,17278.0,17270.0,27662.0,25196.0,24929.0,3227.0,2929.0,2987.0
1,ABB,Amplifier/Comparator,Industrial,Power & Energy,2019,8.471900,AmericasAsia & Oceania (exc. Japan)EMEAJapan,189.490,163733.518,694336.713,56477.645,19018.0,19059.0,17278.0,27978.0,27662.0,25196.0,3347.0,3227.0,2929.0
2,ABB,Amplifier/Comparator,Industrial,Power & Energy,2020,8.661150,AmericasAsia & Oceania (exc. Japan)EMEAJapan,195.081,163007.011,715358.270,58137.779,18123.0,19018.0,19059.0,26134.0,27978.0,27662.0,2668.0,3347.0,3227.0
3,ABB,Amplifier/Comparator,Industrial,Power & Energy,2021,10.090255,AmericasAsia & Oceania (exc. Japan)EMEAJapan,240.245,212172.612,898036.093,67325.066,19407.0,18123.0,19018.0,28945.0,26134.0,27978.0,4641.0,2668.0,3347.0
4,ABB,Amplifier/Comparator,Industrial,Power & Energy,2022,10.040442,AmericasAsia & Oceania (exc. Japan)EMEAJapan,257.635,242620.053,1010453.110,73245.445,19712.0,19407.0,18123.0,29446.0,28945.0,26134.0,4477.0,4641.0,2668.0


In [7]:
df.rename({"Value":"CompanySpend", "Spendings":"TotalSpend"}, axis=1, inplace=True)

In [8]:
df= pd.get_dummies(df, columns=["Device", "Market", "SubMarket"])

In [9]:
df.head()

,Company,Year,CompanySpend,Region,TotalSpend,DevSpends,MarketSpends,SubMarketSpends,Cogs,Cogs_1,...,SubMarket_Other Peripherals,SubMarket_Other Storage,SubMarket_Other Wired,SubMarket_Other Wireless,SubMarket_Power & Energy,SubMarket_STB,SubMarket_TV,SubMarket_Tablet PCs,SubMarket_Test & Measurement,SubMarket_Video Games
0,ABB,2018,7.466000,AmericasAsia & Oceania (exc. Japan)EMEAJapan,200.530,172603.835,744408.882,59122.300,19059.0,17278.0,...,False,False,False,False,True,False,False,False,False,False
1,ABB,2019,8.471900,AmericasAsia & Oceania (exc. Japan)EMEAJapan,189.490,163733.518,694336.713,56477.645,19018.0,19059.0,...,False,False,False,False,True,False,False,False,False,False
2,ABB,2020,8.661150,AmericasAsia & Oceania (exc. Japan)EMEAJapan,195.081,163007.011,715358.270,58137.779,18123.0,19018.0,...,False,False,False,False,True,False,False,False,False,False
3,ABB,2021,10.090255,AmericasAsia & Oceania (exc. Japan)EMEAJapan,240.245,212172.612,898036.093,67325.066,19407.0,18123.0,...,False,False,False,False,True,False,False,False,False,False
4,ABB,2022,10.040442,AmericasAsia & Oceania (exc. Japan)EMEAJapan,257.635,242620.053,1010453.110,73245.445,19712.0,19407.0,...,False,False,False,False,True,False,False,False,False,False


In [10]:
X_train = df[(df.Year<=2022)]
X_test = df[(df.Year>=2023)]
y_train = X_train.pop("CompanySpend")
y_test = X_test.pop("CompanySpend")
X_train.drop(["Company", "Year"], axis=1, inplace=True)
X_test.drop(["Company", "Year"], axis=1, inplace=True)

/var/folders/l1/b9500nvd2v57y822rhwp_f8c0000gp/T/ipykernel_37923/3952824684.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(["Company", "Year"], axis=1, inplace=True)
/var/folders/l1/b9500nvd2v57y822rhwp_f8c0000gp/T/ipykernel_37923/3952824684.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(["Company", "Year"], axis=1, inplace=True)


In [11]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [12]:
model_xgb = xgb.XGBRegressor(verbosity=2, device="cuda")

In [13]:

parameters = {'learning_rate': np.arange(0.001, 0.1, 0.005), 
              'max_depth': [4, 5, 6, 7, 8],
              'min_child_weight': [1, 2, 3, 4, 5],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [100, 200, 300, 500],
              'alpha': np.arange(0, 2, 0.1),
              'lambda' : np.arange(0,2,0.1)}

xgb_grid = RandomizedSearchCV(model_xgb,
                        parameters,
                        cv = 4,
                        n_jobs = 1,
                        verbose=3,
                        n_iter = 500)


In [14]:
xgb_grid.fit(X_train,
         y_train)

Fitting 4 folds for each of 500 candidates, totalling 2000 fits
[CV 1/4] END alpha=1.8, colsample_bytree=0.7, lambda=0.30000000000000004, learning_rate=0.076, max_depth=8, min_child_weight=1, n_estimators=100, subsample=0.7;, score=nan total time=   0.0s
[CV 2/4] END alpha=1.8, colsample_bytree=0.7, lambda=0.30000000000000004, learning_rate=0.076, max_depth=8, min_child_weight=1, n_estimators=100, subsample=0.7;, score=nan total time=   0.0s
[CV 3/4] END alpha=1.8, colsample_bytree=0.7, lambda=0.30000000000000004, learning_rate=0.076, max_depth=8, min_child_weight=1, n_estimators=100, subsample=0.7;, score=nan total time=   0.0s
[CV 4/4] END alpha=1.8, colsample_bytree=0.7, lambda=0.30000000000000004, learning_rate=0.076, max_depth=8, min_child_weight=1, n_estimators=100, subsample=0.7;, score=nan total time=   0.0s
[CV 1/4] END alpha=1.8, colsample_bytree=0.7, lambda=1.9000000000000001, learning_rate=0.041, max_depth=8, min_child_weight=1, n_estimators=200, subsample=0.7;, score=nan t

ValueError: 
All the 2000 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2000 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/tommasosalvatori/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tommasosalvatori/anaconda3/lib/python3.11/site-packages/xgboost/core.py", line 729, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "/Users/tommasosalvatori/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py", line 1051, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tommasosalvatori/anaconda3/lib/python3.11/site-packages/xg

In [ ]:
best_params = xgb_grid.best_params_
best_score = xgb_grid.best_score_

In [ ]:
best_params

{'subsample': 0.7,
 'n_estimators': 200,
 'min_child_weight': 2,
 'max_depth': 4,
 'learning_rate': 0.001,
 'lambda': 0.5,
 'colsample_bytree': 0.7,
 'alpha': 0.30000000000000004}

In [ ]:
best_score

-0.03734267958655807

In [ ]:
new_xgb = xgb_grid.best_estimator_

In [ ]:
from sklearn.metrics import r2_score
r2_score(new_xgb.predict(X_test), y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor

model_cart = DecisionTreeRegressor()
model_cart.fit(X_train, y_train)
r2_score(model_cart.predict(X_train), y_train)

In [ ]:
r2_score(y_test, model_cart.predict(X_test))

In [ ]:
n = 0
d = 0
y_pred =model_cart.predict(X_test)
y_train_mean = y_train.mean()
for i in range(len(y_test)):
    n += (y_test.iloc[i] - y_pred[i]) ** 2
    d += (y_test.iloc[i] - y_train_mean) ** 2
r_squared = 1 - n/d
print(r_squared)

In [ ]:
y_pred=model_cart.predict(X_test)

In [ ]:
new_df=pd.DataFrame({"pred":y_pred, "test":y_test})

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(25,20))
plot_tree(model_cart)
plt.show()

In [ ]:
model_cart2 = DecisionTreeRegressor(max_depth=10, min_samples_leaf=5)
model_cart2.fit(X_train, y_train)
r2_score(model_cart2.predict(X_train), y_train)

In [ ]:
n = 0
d = 0
y_pred2 =model_cart2.predict(X_test)
y_train_mean = y_train.mean()
for i in range(len(y_test)):
    n += (y_test.iloc[i] - y_pred2[i]) ** 2
    d += (y_test.iloc[i] - y_train_mean) ** 2
r_squared2 = 1 - n/d
print(r_squared2)

In [ ]:
fig = plt.figure(figsize=(25,20))
plot_tree(model_cart2)
plt.show()